In [1]:
import os
os.getcwd()

'/home/onyxia/work/hackathon-un-2022/notebooks'

In [2]:
os.chdir("..")
import utils.functions as fc

In [3]:
from importlib import reload
fc = reload(fc)

In [4]:
ship_data_enriched = fc.create_ship_data_enriched()

In [5]:
AIS = fc.read_ais_parquet()

In [6]:
AIS.shape

(1543295, 9)

In [7]:
ship_data_enriched = ship_data_enriched.loc[ship_data_enriched["ShiptypeLevel1"] == "Cargo Carrying",:]

In [8]:
AIS.shape

(1543295, 9)

In [31]:
# ship_data_enriched = ship_data_enriched.loc[ship_data_enriched["ShipTypeLevel3"].isin(["General Cargo",
#                                                                                       "Bulk Dry", 
#                                                                                      "Container", 
#                                                                                       "Other Bulk Dry",
#                                                                                       "Refrigerated Cargo",
#                                                                                       "Other Dry Cargo",
#                                                                                       "Bulk Dry / Oil",
#                                                                                       "Self Discharging Bulk Dry"]),:]

In [9]:
AIS.loc[AIS["mmsi"].isna(), :].shape

(0, 9)

In [10]:
AIS_enriched = fc.enrich_AIS_data(
    AIS, ship_data_enriched
)

In [11]:
AIS_enriched.shape

(585145, 128)

In [12]:
AIS_enriched.columns

Index(['hex_resolution', 'longitude', 'mmsi', 'destination', 'eeid',
       'H3_int_index_8', 'latitude', 'dt_insert_utc', 'polygon_name',
       'LRIMOShipNo',
       ...
       'ShipTypeLevel4', 'Level3Code', 'ShipTypeLevel3', 'Level2Code',
       'ShipTypeLevel2', 'ShipTypeLevel1Code', 'ShiptypeLevel1', 'HullType',
       'SubGroup', 'SubType'],
      dtype='object', length=128)

In [21]:
# function qui assigne le pays d'origine
import pandas as pd
def assign_country_origin(AIS_enriched):
    mmsi_number = pd.read_csv(fs.open(f'{BUCKET}/AIS/mmid.csv',
                         mode='rb'), error_bad_lines=False, encoding='iso-8859-1', sep=";")
    AIS_enriched["Digit"] = AIS_enriched["mmsi"].astype(str).str[0:3]
    mmsi_number["Digit"] = mmsi_number["Digit"].astype(str)
    AIS_enriched = pd.merge(AIS_enriched, mmsi_number, on=["Digit"], how="left")
    AIS_enriched = AIS_enriched.rename({"Allocated to" : "origin_country"}, axis=1)
    return AIS_enriched


In [22]:
fs = fc.create_s3_fs()
BUCKET = "projet-hackathon-un-2022"

In [23]:
AIS_enriched2 = assign_country_origin(AIS_enriched)

/tmp/ipykernel_799/2296097229.py:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  mmsi_number = pd.read_csv(fs.open(f'{BUCKET}/AIS/mmid.csv',


In [ ]:
def assign_destination_country():
    

In [40]:
mmsi_number.head()

,Digit,Allocated to
0,201,Albania (Republic of)
1,202,Andorra (Principality of)
2,203,Austria
3,204,Portugal - Azores
4,205,Belgium


In [41]:
AIS_enriched["Digit"] = AIS_enriched["mmsi"].astype(str).str[0:3]

In [43]:
AIS_enriched["Digit"].head()

0    272
1    272
2    272
3    272
4    272
Name: Digit, dtype: object

In [44]:
mmsi_number["Digit"] = mmsi_number["Digit"].astype(str)

In [45]:
AIS_enriched2 = pd.merge(AIS_enriched, mmsi_number, on=["Digit"], how="left")

In [46]:
AIS_enriched2.head()

,hex_resolution,longitude,mmsi,destination,eeid,H3_int_index_8,latitude,dt_insert_utc,polygon_name,LRIMOShipNo,...,ShipTypeLevel3,Level2Code,ShipTypeLevel2,ShipTypeLevel1Code,ShiptypeLevel1,HullType,SubGroup,SubType,Digit,Allocated to
0,8,31.408333,272157700,MYKOLAIV,5191743282127358980,613021963599740927,47.526667,2022-01-01 05:58:49,Polygon,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272,Ukraine
1,8,31.331667,272157700,MYKOLAIV,5191743282127358980,613021966030340095,47.535000,2022-01-01 18:25:52,Polygon,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272,Ukraine
2,8,31.331667,272157700,MYKOLAIV,5191743282127358980,613021966030340095,47.535000,2022-01-01 21:14:30,Polygon,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272,Ukraine
3,8,31.333333,272157700,MYKOLAIV,5191743282127358980,613021966030340095,47.535000,2022-01-01 07:46:57,Polygon,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272,Ukraine
4,8,31.333333,272157700,MYKOLAIV,5191743282127358980,613021966030340095,47.535000,2022-01-01 09:43:50,Polygon,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272,Ukraine


In [48]:
AIS_enriched2.loc[AIS_enriched2["destination"].isna(), :].shape[0]/AIS_enriched2.shape[0] # 377 313, 24 %

0.24448533818874552

In [49]:
AIS_enriched2.loc[~AIS_enriched2["destination"].isna(), :].shape # 1 165 982

(1165982, 130)

In [50]:
AIS_enriched2 = AIS_enriched2.rename({"Allocated to" : "origin_country"}, axis=1)

In [51]:
AIS_enriched2["origin_country"].value_counts()

Russian Federation                                                               296823
Ukraine                                                                          233616
Romania                                                                          205072
Republic of Türkiye                                                              138906
Panama (Republic of)                                                              96314
                                                                                  ...  
Syrian Arab Republic                                                                 90
Seychelles (Republic of)                                                             76
Belgium                                                                               4
Japan                                                                                 1
United Kingdom of Great Britain and Northern Ireland - British Virgin Islands         1
Name: origin_country, Length: 71

In [52]:
AIS_enriched2.loc[AIS_enriched2["origin_country"]=="",:].shape

(0, 130)

In [53]:
AIS_enriched2.loc[AIS_enriched2["origin_country"].isna(), :].shape

(0, 130)

In [56]:
ports = pd.read_csv('https://msi.nga.mil/api/publications/download?type=view&key=16920959/SFH00000/UpdatedPub150.csv')

In [57]:
ports.head()

,World Port Index Number,Region Name,Main Port Name,Alternate Port Name,UN/LOCODE,Country Code,World Water Body,IHO S-130 Sea Area,Sailing Direction or Publication,Standard Nautical Chart,...,Supplies - Fuel Oil,Supplies - Diesel Oil,Supplies - Aviation Fuel,Supplies - Deck,Supplies - Engine,Repairs,Dry Dock,Railway,Latitude,Longitude
0,4580,US Lake Michigan -- 4570,St. James Harbor,,,United States,Lake Michigan; Great Lakes,,U.S. Coast Pilot 6 - Great Lakes and their con...,14911,...,Yes,Yes,Unknown,Yes,Yes,Emergency Only,Unknown,Unknown,45.733333,-85.516667
1,63110,Antarctica -- 63060,Melchior Harbor,,,Antarctica,South Pacific Ocean,,Sailing Directions Pub. 200 (Planning Guide & ...,29106,...,Unknown,Unknown,Unknown,Unknown,Unknown,None,Unknown,Unknown,-64.316667,-63.000000
2,52350,Sulawesi -- 51970,Donggala,,,Indonesia,Makassar Strait; South Pacific Ocean,,Sailing Directions Pub. 163 (Enroute) - Borneo...,72171,...,No,No,Unknown,Unknown,Unknown,Emergency Only,Unknown,Unknown,-0.650000,119.733333
3,46544,Angola -- 46540,Psvm Terminal,,,Angola,South Atlantic Ocean,,Sailing Directions Pub. 123 (Enroute) - Southw...,57240,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,-6.250000,10.733333
4,56820,Bismarck Archipelago -- 56700,Kokopo,,,Papua New Guinea,Bismarck Sea; South Pacific Ocean,,Sailing Directions Pub. 126 (Enroute) - Pacifi...,82192,...,No,No,Unknown,No,No,Limited,Unknown,Unknown,-4.333333,152.266667


In [58]:
ports = ports.loc[:, ["Main Port Name", "Country Code"]]

In [59]:
ports.head()

,Main Port Name,Country Code
0,St. James Harbor,United States
1,Melchior Harbor,Antarctica
2,Donggala,Indonesia
3,Psvm Terminal,Angola
4,Kokopo,Papua New Guinea


In [60]:
AIS_enriched2["destination"].value_counts()

CONSTANTA            98262
NOVOROSSIYSK         67560
RU NVS               38405
GALATI               29892
VARNA                29436
                     ...  
RU 454                   1
IT-GAI                   1
ES BCN>FOR ORDERS        1
P!.#%6/                  1
URK                      1
Name: destination, Length: 1085, dtype: int64

In [63]:
ports["Main Port Name"] = ports["Main Port Name"].str.upper()

In [64]:
ports["Main Port Name"].head()

0    ST. JAMES HARBOR
1     MELCHIOR HARBOR
2            DONGGALA
3       PSVM TERMINAL
4              KOKOPO
Name: Main Port Name, dtype: object

In [65]:
ports = ports.rename({"Main Port Name" : "destination"}, axis=1)

In [66]:
AIS_enriched3 = pd.merge(AIS_enriched2, ports, on = ["destination"], how="left")

In [67]:
AIS_enriched3.shape

(1543310, 131)

In [68]:
AIS_enriched3 = AIS_enriched3.rename({"Country Code": "destination_country"}, axis=1)

In [69]:
AIS_enriched3["destination_country"].value_counts()

Romania          149219
Russia            84160
Ukraine           45783
Bulgaria          39590
Turkey            35536
Georgia           14124
Netherlands        1385
Israel              779
Italy               595
Bangladesh          539
Portugal            512
Somalia             319
South Africa        230
Greece              195
Croatia             187
United States       184
China               163
Slovenia            142
Spain               131
Algeria             106
South Korea          85
Cyprus               78
Egypt                45
Colombia             15
Name: destination_country, dtype: int64

In [ ]:
# a present imputation de la destination pour les 